# Black Scholes Model

The Black-Scholes Model was developed by professor Fisher Black, Myron Scholes, and Robert Merton. The model is used to value option contracts. The model makes the following assumptions: 
- Lognormal distribution for the stock price
- There are no transaction costs or taxes
- There is no cash flow from the underlying asset during the time to maturity of the option
- There are no arbitrage opportunities
- Investors can borrow or lend at the risk-free rate 
- The risk-free rate is constant
- The model is based on European-style options


The key concept from the mathematical model is that the option and the stock price depend on the same underlying source of uncertainty. This uncertainty could be eliminated by constructing a portfolio with the stock and the option. This portfolio should be instantaneously riskless and earn the risk-free rate. The portfolio will require continuous rebalancing.

The mathematical formula to calculate the call price of a European option is:

\begin{equation*}
C = S_t N \left(d1 \right) - K e^{-rt} N \left(d2 \right)
\end{equation*}

__Where:__

\begin{equation*}
d1 = \frac{ \ln{ \frac{S_t}{K} + \left(r + \frac{\sigma_u^2}{2} \right) t} }{\sigma_s \sqrt{t}}
\end{equation*}

and:

\begin{equation*}
d2 = d1 -  \sigma_s \sqrt{t}
\end{equation*}


__Where:__

\begin{alignat*}{1}
&C \quad & = \quad & \text{Call option price} \\
&S \quad & = \quad & \text{Current stock (or other underlying) price} \\
&K \quad & = \quad & \text{Strike price} \\
&r \quad & = \quad & \text{Risk-free interest rate} \\
&t \quad & = \quad & \text{time to maturity} \\
&N \quad & = \quad & \text{A normal distribution}
\end{alignat*}

In this notebook, I will develop a process to calculate the call price of any option given all the parameters required in the Black-Scholes model. To do that, I will use Python, specifically Yfinance and Quandl, libraries to obtain the required data.

First, let's import the necessary libraries to make the analysis.

__Note:__ Some libraries might not be pre-installed so the reader will need to uncomment the line code to install them. 

In [1]:
#pip install stockquotes
#pip install Quandl

import math
import scipy.stats as st
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import stockquotes
import quandl
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from BlackScholesMerton import BlackScholesModel
import os

%matplotlib inline

I will use the Quandl library to obtain information about the risk-free rate. To use Quandl, you will need to set up an account and get an API key. The code to call Quandl API is <code>quandl.ApiConfig.api_key = 'YOUR KEY GOES HERE'</code>

In [2]:
quandl.ApiConfig.api_key = os.environ.get('QUANDL_KEY')

From the YFinance library, I will get the historical information of the S&P500.

In [3]:
GSPC = yf.Ticker('^GSPC')

This library also allows getting the expiration dates for the options.

In [4]:
expirations = GSPC.options
expirations

('2020-12-17', '2021-12-16')

Based on the expirations, I can get the option chain with the information required to calculate the option price using the Black-Scholes model. Let's have a look at the information available for the option chain on the S&P 500 with expiration on December 17, 2020.

In [5]:
GSPC_opts = GSPC.option_chain(expirations[0])
GSPC_opts

Options(calls=        contractSymbol       lastTradeDate  strike  lastPrice      bid  \
0   SPX201218C00100000 2020-07-14 13:51:29   100.0    3011.56  3095.90   
1   SPX201218C00200000 2020-06-18 16:31:43   200.0    2878.00  2996.00   
2   SPX201218C00300000 2020-07-10 00:10:28   300.0    2738.00  2896.40   
3   SPX201218C00400000 2020-07-10 00:10:30   400.0    2638.40  2797.30   
4   SPX201218C00500000 2020-07-10 00:10:30   500.0    1790.50  2691.70   
..                 ...                 ...     ...        ...      ...   
89  SPX201218C03800000 2020-07-17 19:15:15  3800.0      14.00    13.60   
90  SPX201218C03900000 2020-07-17 17:59:54  3900.0       8.90     8.70   
91  SPX201218C04000000 2020-07-17 18:18:23  4000.0       6.00     5.60   
92  SPX201218C04100000 2020-07-17 15:02:30  4100.0       3.91     3.70   
93  SPX201218C04200000 2020-07-17 17:45:02  4200.0       2.70     2.55   

        ask  change  percentChange  volume  openInterest  impliedVolatility  \
0   3099.00    0.0

This analysis will focus only on calculating the call price; however, once a put price formula is defined the reader can easily run the code with all the variables already defined in this notebook. Let's have a look at the DataFrame generated from YFinance.

In [6]:
GSPC_calls = GSPC_opts.calls
GSPC_calls

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,SPX201218C00100000,2020-07-14 13:51:29,100.0,3011.56,3095.90,3099.00,0.00,0.000000,345,3263,0.000010,True,REGULAR,USD
1,SPX201218C00200000,2020-06-18 16:31:43,200.0,2878.00,2996.00,2999.20,0.00,0.000000,73,3137,0.000010,True,REGULAR,USD
2,SPX201218C00300000,2020-07-10 00:10:28,300.0,2738.00,2896.40,2899.50,0.00,0.000000,8,86,0.000010,True,REGULAR,USD
3,SPX201218C00400000,2020-07-10 00:10:30,400.0,2638.40,2797.30,2800.40,0.00,0.000000,6,16,0.000010,True,REGULAR,USD
4,SPX201218C00500000,2020-07-10 00:10:30,500.0,1790.50,2691.70,2694.90,0.00,0.000000,0,59,0.000010,True,REGULAR,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,SPX201218C03800000,2020-07-17 19:15:15,3800.0,14.00,13.60,14.10,-1.60,-10.256412,399,14534,0.178273,False,REGULAR,USD
90,SPX201218C03900000,2020-07-17 17:59:54,3900.0,8.90,8.70,9.10,0.15,1.714281,5,7338,0.180771,False,REGULAR,USD
91,SPX201218C04000000,2020-07-17 18:18:23,4000.0,6.00,5.60,6.00,0.30,5.263161,24,7882,0.184136,False,REGULAR,USD
92,SPX201218C04100000,2020-07-17 15:02:30,4100.0,3.91,3.70,4.10,0.10,2.624676,4,2025,0.188469,False,REGULAR,USD


From this dataFrame, I need the information related to the strike price and the implied volatility.

We can save the strike price in a dictionary where the keys will be the different expiration dates and the values will be an array with all the strike prices available on the specific expiration date. 

In [7]:
K = {name: pd.DataFrame()for name in expirations}

for i, a in enumerate(expirations):
    GSPC_opts = GSPC.option_chain(expirations[i])
    GSPC_calls = GSPC_opts.calls
    K[a] = GSPC_calls['strike']

Similarly, I can get a dictionary with information related to the implied volatility. Once again, the keys of the dictionary will be the different expiration dates and the values of the dictionary will be an array with the implied volatility corresponding to a determined strike price.

In [8]:
sigma = {name: pd.DataFrame() for name in expirations}

for i, a in enumerate(expirations):
    GSPC_opts = GSPC.option_chain(expirations[i])
    GSPC_calls = GSPC_opts.calls
    sigma[a] = GSPC_calls['impliedVolatility']

With the same logic used before, I can get the time to maturity for each expiration date. I will define a dictionary where the keys are the different expiration dates available, and the values of the dictionary are the difference between the current date and the expiration date. However, in this case, values are not an array of data, instead, the values are a scalar.

In [10]:
t = {name: pd.DataFrame() for name in expirations}

for i, a in enumerate(expirations):
    expiration_date = datetime.strptime(expirations[i],"%Y-%m-%d") 
    today = datetime.now()
    t[a] = (expiration_date - today).days/365

Using the Quandl library, I can get access to the daily Treasury Yield Curve and saved it in a variable called <code>yield_curve</code>. To get the yield curve, I used the get module as follows: <code>quandl.get("USTREASURY/YIELD", authtoken="YOUR TOKEN GOES HERE")</code>

In [ ]:
yield_curve = quandl.get("USTREASURY/YIELD", authtoken = os.environ.get('QUANDL_KEY'))
yield_curve

Now, I will define a function that allows me to calculate the time to maturity of the option based on the expiration date. The next step will be to match this time to maturity with the most appropriate treasury yield.

To achieve this matching task, I will define a function that browses through the list of available treasury maturities and matches the time to maturity of the option with the corresponding yield of the closest greater treasury maturity.

In [ ]:
def TTM(exp_date,today):
    time_to_maturity = (exp_date - today).days
    return time_to_maturity

def rf(exp_date,today):
    num_days = [30,60,90,180,365,730,1095,1825,2555,3650,7300,10950]
    rf_ttm = list(yield_curve.columns)
    risk_free_L = []
    if round(TTM(exp_date,today)) < 0:
        return "Expire date must be greater than today"
    elif round(TTM(exp_date,today)) > num_days[-1]: 
        return yield_curve[rf_ttm[-1]][-1]/100
    else:
        for b, f in zip(num_days, rf_ttm):
                if round(TTM(expiration_date,today)) < b:
                    risk_free_L.append(yield_curve[f][-1]/100)
        return risk_free_L[0]

With this last function defined, I can create a dictionary with the different risk-free rates relevant for each expiration. Again, the keys of the dictionary will be the expirations of the option and the values will be a scalar with the corresponding risk-free rate.

In [ ]:
rf_rate = {name: pd.DataFrame() for name in expirations}

for i, a in enumerate(expirations):
    expiration_date = datetime.strptime(expirations[i],"%Y-%m-%d") 
    today = datetime.now()
    rf_rate[a] = rf(expiration_date,today)

The last variable needed to calculate the price of the call option is the current price of the security. This can be obtained from the stockquotes library through the <code>Stock</code> module.

In [11]:
SP500 = stockquotes.Stock("^GSPC")
S = SP500.current_price

Now, let's develop the Black-Scholes model. I will define the model with 3 different functions; the advantage of this is that these functions could potentially  be used to create a Black-Scholes Class in Python for future development projects.

The first function will calculate the expected value of the stock if the option is exercised using risk-adjusted probabilities. This is the equivalent of the term $\begin{equation*}N \left(d1 \right)\end{equation*}$. The function will return a tuple containing $\begin{equation*}d1 \end{equation*}$ and $\begin{equation*}N \left(d1 \right)\end{equation*}$.

In [ ]:
def Norm_d1(S, K, r, t, sigma):
    d1 = (np.log(S/K)+(r+((np.power(sigma,2)/2))*t))/(sigma*math.sqrt(t))
    N_d1 = st.norm.cdf(d1)
    return (d1, N_d1)

The next function that I will define is the probability of the function to be exercised based on risk-adjusted probabilities. This is equivalent to the term $\begin{equation*}N \left(d2 \right)\end{equation*}$.

In [ ]:
def Norm_d2(d1, sigma, t):
    d2 = d1 - sigma*math.sqrt(t)
    N_d2 = st.norm.cdf(d2)
    return N_d2

Lastly, I will define the function that will calculate the call price taking an input all the variables required in the model and using the previous two functions.

In [ ]:
def Call_price(S, K, r, t, sigma):
    d1, N_d1 = Norm_d1(S, K, r, t, sigma)
    N_d2 = Norm_d2(d1, sigma, t)
    C = S*N_d1-K*math.exp(-r*t)*N_d2
    return C

With the call price function define, I can create a dictionary that saves the array of call prices for different expiration dates. 

In [ ]:
C = {name: pd.DataFrame() for name in expirations}

for i, a in enumerate(expirations):
    C[a] = Call_price(S, K[a], rf_rate[a], t[a], sigma[a])

Let's visualize the results. The following graph will compare, for the two expirations available, the call price for different strike prices calculated using the Black-Scholes model and the actual price taken from the YFinance library. 

In [ ]:
def callP_plot(n_rows, n_col, figure_size, expir_dates, subtitle=None):
    
    if not isinstance(figure_size, (list,tuple)):
        print("Figure size has to be Tuple or List with the width and the height")
        return

    fig,axs = plt.subplots(n_rows,n_col,figsize=figure_size)

    if subtitle == None:
        name = GSPC.info['shortName']
        fig.suptitle(f'{name} Call Value vs. Strike Price',fontsize=18,fontweight='bold')
    else:
        fig.suptitle(subtitle,fontsize=18,fontweight='bold')
    i, j = 0, 0
    for n in (0,len(expir_dates)-1):
        
        if n_rows == 1:
            ax = axs[j]
        elif n_col == 1:
            ax = axs[i]
        else:
            ax = axs[i,j]

        ax.plot(K[expir_dates[n]], 
                    C[expir_dates[n]],
                    'tab:red',
                    linewidth=4.0,
                    label='BSM model') #Plots the first expiration based on BSM model
        ax.plot(K[expir_dates[n]], 
                    GSPC.option_chain(expir_dates[n]).calls['lastPrice'],
                    'tab:cyan',
                    linewidth=1.0, 
                    label='Market price') #Plots the first expiration based on actual market price
        ax.legend(labelspacing=2,borderpad=1.25,fontsize=10) #Set legend 
        ax.set_xlim(xmin=0.0) #To start the graph on (0,0)
        ax.set_ylim(ymin=0.0)
        ax.set_xlabel("Strike Price",fontsize=14) #Set X-axis label
        ax.set_ylabel("Call Price",fontsize=14) #Set Y-axis label
        ax.set_title(f"Expiration: {datetime.strptime(expir_dates[n],'%Y-%m-%d').strftime('%b %d, %Y')}",
                         fontsize=14) #Subtitle on the first figure
        ax.spines['right'].set_visible(False) #Remove right border on the first subplot
        ax.spines['top'].set_visible(False)  #Remove top border on the first subplot
        
        if i + 1 > n_rows - 1:
            j = j + 1
            i = 0
        else:
            i += 1
    plt.show()

    plt.close()    

In [ ]:
callP_plot(1,2,(20,7),expir_dates = expirations)

The difference between the Black-Scholes model and the actual price could be explained by lack of liquidity, especially in options with low strike price, and/or differences in the implied volatility. 

__References:__<p>&nbsp;</p> 
- Investopedia (2020). Black Scholes Model https://www.investopedia.com/terms/b/blackscholes.asp
- John C. Hull (2016). Fundamentals of Futures and Options Markets, 9th Ed, Ch 13

# Implied Volatility

In [ ]:
expiration_date = datetime.strptime(expirations[1],"%Y-%m-%d") 
td = datetime.now()

In [ ]:
GSPC_opts = GSPC.option_chain(expirations[0])

GSPC_calls = GSPC_opts.calls

expiration_date = datetime.strptime(expirations[1],"%Y-%m-%d")
td = datetime.now()
strike = GSPC_calls['strike'][1]
print(strike)

SP500 = stockquotes.Stock("^GSPC")
stock_price = SP500.current_price

implied_vol = GSPC_calls['impliedVolatility'][1]

In [ ]:
BSM = BlackScholesModel(expiration_date, td,stock_price,strike, implied_vol)

print(BSM.risk_free())

In [ ]:
print(BSM.maturity())

In [ ]:
print(BSM.norm_d1())

In [ ]:
print(BSM.norm_d2())

In [ ]:
print(BSM.call_price())

In [ ]:
BSM.callp_plot(1,1,(20,7))